In [7]:
# Scientic libraries
import numpy as np
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
import logging

# Helper libraries 
from tqdm.notebook import tqdm,trange
import warnings 
warnings.filterwarnings('ignore')

%config InlineBackend.figure_format = 'retina' # sets the figure format to 'retina' for high-resolution displays.

# Pandas options
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' # To display all interaction 
pd.set_option('display.max_columns', None)


# Table styles
table_styles = {
    'cerulean_palette': [
        dict(selector="th", props=[("color", "#FFFFFF"), ("background", "#004D80")]),
        dict(selector="td", props=[("color", "#333333")]),
        dict(selector="table", props=[("font-family", 'Arial'), ("border-collapse", "collapse")]),
        dict(selector='tr:nth-child(even)', props=[('background', '#D3EEFF')]),
        dict(selector='tr:nth-child(odd)', props=[('background', '#FFFFFF')]),
        dict(selector="th", props=[("border", "1px solid #0070BA")]),
        dict(selector="td", props=[("border", "1px solid #0070BA")]),
        dict(selector="tr:hover", props=[("background", "#80D0FF")]),
        dict(selector="tr", props=[("transition", "background 0.5s ease")]),
        dict(selector="th:hover", props=[("font-size", "1.07rem")]),
        dict(selector="th", props=[("transition", "font-size 0.5s ease-in-out")]),
        dict(selector="td:hover", props=[('font-size', '1.07rem'),('font-weight', 'bold')]),
        dict(selector="td", props=[("transition", "font-size 0.5s ease-in-out")])
    ]
}

# Seed value for numpy.random => makes notebooks stable across runs
np.random.seed(42)

In [4]:
df = pd.read_csv('data/loan_data_2007_2014.csv')

In [14]:
pd.set_option('display.max_info_columns', 100)
pd.set_option('display.max_rows', 100)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 466285 entries, 0 to 466284
Data columns (total 75 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Unnamed: 0                   466285 non-null  int64  
 1   id                           466285 non-null  int64  
 2   member_id                    466285 non-null  int64  
 3   loan_amnt                    466285 non-null  int64  
 4   funded_amnt                  466285 non-null  int64  
 5   funded_amnt_inv              466285 non-null  float64
 6   term                         466285 non-null  object 
 7   int_rate                     466285 non-null  float64
 8   installment                  466285 non-null  float64
 9   grade                        466285 non-null  object 
 10  sub_grade                    466285 non-null  object 
 11  emp_title                    438697 non-null  object 
 12  emp_length                   445277 non-null  object 
 13 

### Pre-processing continous variables

In [17]:
df['emp_length'].unique()

array(['10+ years', '< 1 year', '1 year', '3 years', '8 years', '9 years',
       '4 years', '5 years', '6 years', '2 years', '7 years', nan],
      dtype=object)

In [21]:
df['emp_length']=df['emp_length'].str.replace('+ years','')
df['emp_length']=df['emp_length'].str.replace('< 1 year',str(0))

In [25]:
df['emp_length'].fillna('0',inplace=True)

In [30]:
df['emp_length']=df['emp_length'].str.replace(' years','')

In [33]:
df['emp_length']=df['emp_length'].str.replace(' year','')

In [34]:
df['emp_length'].unique()

array(['10', '0', '1', '3', '8', '9', '4', '5', '6', '2', '7'],
      dtype=object)

In [38]:
# Converting string to numeric
df['emp_length']=df['emp_length'].astype('int')

In [ ]:
df['earliest_cr_line']=pd.to_datetime(df['earliest_cr_line'], format='%b-%y')

In [48]:
# Having abosulte dates is not useful
# Sinc we are doing analysis in future
# We have to take date which is greater than 2014
# Let's assume we are doing analysis in december 2017

df['days_since_earliest_cr_line']=pd.to_datetime('2017-12-01') - df['earliest_cr_line']

In [54]:
df['days_since_earliest_cr_line']

0        12022 days
1         6819 days
2         5874 days
3         7974 days
4         8005 days
            ...    
466280    5358 days
466281    7488 days
466282    5844 days
466283    5417 days
466284    6513 days
Name: days_since_earliest_cr_line, Length: 466285, dtype: timedelta64[ns]

In [62]:
# Similarly let's take months sinc earliest cr line
df['mnths_since_earliest_cr_line']=((pd.to_datetime('2017-12-01') - df['earliest_cr_line']) / pd.to_timedelta(1, 'D')).apply(lambda x: x // 30)

In [ ]:
# If you can see the number of months is in negative
df['mnths_since_earliest_cr_line'].describe()

count    466256.000000
mean        242.497130
std          95.353324
min        -621.000000
25%         185.000000
50%         228.000000
75%         289.000000
max         595.000000
Name: mnths_since_earliest_cr_line, dtype: float64

In [64]:
# This cannot be negative, but when converting Jan-68 to datetime this might have become 2068 rather than 1968
# So what can we do to solve this issue? The soln is replace these with max because those are very old dates

In [73]:
df.loc[df['mnths_since_earliest_cr_line'] < 0, 'mnths_since_earliest_cr_line'] = df['mnths_since_earliest_cr_line'].max()

In [ ]:
# Prprocessing Term
df['term'].unique()

array([' 36 months', ' 60 months'], dtype=object)

In [80]:
df['term']=df['term'].str.strip().str.replace(' months','')

In [83]:
df['term']=df['term'].astype('int')

In [86]:
df['issue_d_date']=pd.to_datetime(df['issue_d'],format='%b-%y')

In [89]:
df['mnths_since_issue_d']=((pd.to_datetime('2017-12-01')-df['issue_d_date'])/pd.Timedelta(1,'D')).apply(lambda x: x//30)